In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
import torch
import torch.nn as nn
import torch.optim as optim

# For Progress Bar
from tqdm import tqdm

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
print(device)


cuda


In [ ]:
# Load MNIST dataset
mnist = fetch_openml('mnist_784')
X = mnist.data.astype('float32') / 255.
y = mnist.target.astype('int')

/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [ ]:
X = np.array(X)
y = np.array(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Perform PCA analysis on the images
pca = PCA(n_components=20)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

# Visualize the first 20 PCA modes
fig, axes = plt.subplots(nrows=4, ncols=5, figsize=(10, 8))
for i, ax in enumerate(axes.flat):
    ax.imshow(pca.components_[i].reshape(28, 28), cmap='gray')
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_title('PC %d' % (i+1))
plt.tight_layout()
plt.show()


In [ ]:
print(X_train_pca.shape)

(56000, 20)


In [ ]:
# Convert data to PyTorch tensors
X_train_pca = torch.from_numpy(X_train_pca).float()
X_test_pca = torch.from_numpy(X_test_pca).float()
y_train = torch.from_numpy(y_train).long()
y_test = torch.from_numpy(y_test).long()

In [ ]:
# Define the neural network architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(20, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 10)

    def forward(self, x):
        x = self.fc1(x)
        x = nn.functional.dropout(x, p=0.2)
        x = nn.functional.relu(self.fc2(x))
        x = nn.functional.dropout(x, p=0.2)
        x = nn.functional.relu(self.fc3(x))
        x = nn.functional.dropout(x, p=0.2)
        x = self.fc4(x)
        return nn.functional.log_softmax(x, dim=1)

# Initialize the neural network
model = Net().to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

# Train the neural network
for epoch in tqdm(range(10000), desc='Training', unit='epoch', unit_scale=True, ncols=80, bar_format='{l_bar}{bar:30}{r_bar}{bar:-10b}'):
    optimizer.zero_grad()
    output = model(X_train_pca.to(device))
    loss = criterion(output, y_train.to(device))
    loss.backward()
    optimizer.step()
    if epoch % 100 ==0:
      print(" Epoch: {:d}, Loss: {:.4f}".format(epoch+1, loss.item()))

# Evaluate the model on the test set
with torch.no_grad():
    output = model(X_test_pca.to(device))
    y_pred = torch.argmax(output, dim=1)
    accuracy = accuracy_score(y_test, y_pred.cpu())
    print("Accuracy: {:.2f}%".format(accuracy * 100))


Training:   0%|                              | 15.0/10.0k [00:00<02:20, 71.2epoc

 Epoch: 1, Loss: 2.3094


Training:   1%|▎                             | 113/10.0k [00:02<03:14, 50.8epoch

 Epoch: 101, Loss: 0.3254


Training:   2%|▋                             | 218/10.0k [00:05<02:37, 62.2epoch

...

 Epoch: 9801, Loss: 0.0627


Training:  99%|█████████████████████████████▋| 9.91k/10.0k [02:55<00:00, 90.6epo

 Epoch: 9901, Loss: 0.0654


Training: 100%|██████████████████████████████| 10.0k/10.0k [02:57<00:00, 56.4epo

Accuracy: 96.26%


In [ ]:
# Reshape the data for the LSTM network
X_train_lstm = X_train.reshape((X_train.shape[0], 28, 28))
X_test_lstm = X_test.reshape((X_test.shape[0], 28, 28))

X_train_lstm = torch.tensor(X_train_lstm, dtype = torch.float32,  requires_grad = True)
X_test_lstm = torch.tensor(X_test_lstm, dtype = torch.float32,  requires_grad = True)
class LSTMNet(nn.Module):
    def __init__(self):
        super(LSTMNet, self).__init__()
        self.lstm1 = nn.LSTM(28, 128, batch_first=True)
        self.fc1 = nn.Linear(128, 10)

    def forward(self, x):
        x, _ = self.lstm1(x)
        x = x[:, -1, :]
        x = self.fc1(x)
        return nn.functional.log_softmax(x, dim=1)

# Initialize the LSTM network
lstm_model = LSTMNet()

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(lstm_model.parameters())

# Train the LSTM network
for epoch in range(20):
    optimizer.zero_grad()
    output = lstm_model(X_train_lstm)
    loss = criterion(output, y_train)
    loss.backward()
    optimizer.step()
    print("Epoch: {:d}, Loss: {:.4f}".format(epoch+1, loss.item()))

# Evaluate the LSTM model on the test set
with torch.no_grad():
    output = lstm_model(X_test_lstm)
    y_pred = torch.argmax(output, dim=1)
    accuracy = accuracy_score(y_test, y_pred)
    print("LSTM Accuracy: {:.2f}%".format(accuracy * 100))


Epoch: 1, Loss: 2.3019
Epoch: 2, Loss: 2.3004
Epoch: 3, Loss: 2.2988
Epoch: 4, Loss: 2.2972
Epoch: 5, Loss: 2.2955
Epoch: 6, Loss: 2.2936
Epoch: 7, Loss: 2.2915
Epoch: 8, Loss: 2.2890
Epoch: 9, Loss: 2.2862
Epoch: 10, Loss: 2.2828
Epoch: 11, Loss: 2.2788
Epoch: 12, Loss: 2.2739
Epoch: 13, Loss: 2.2680
Epoch: 14, Loss: 2.2608
Epoch: 15, Loss: 2.2519
Epoch: 16, Loss: 2.2411
Epoch: 17, Loss: 2.2278
Epoch: 18, Loss: 2.2113
Epoch: 19, Loss: 2.1906
Epoch: 20, Loss: 2.1659
LSTM Accuracy: 24.44%


In [ ]:
# Train a SVM classifier
svm_model = SVC()
svm_model.fit(X_train_pca, y_train)
y_pred = svm_model.predict(X_test_pca)
accuracy = accuracy_score(y_test, y_pred)
print("SVM Accuracy: {:.2f}%".format(accuracy * 100))


SVM Accuracy: 97.37%


In [ ]:
# Train a decision tree classifier
tree_model = DecisionTreeClassifier()
tree_model.fit(X_train_pca, y_train)
y_pred = tree_model.predict(X_test_pca)
accuracy = accuracy_score(y_test, y_pred)
print("Decision Tree Accuracy: {:.2f}%".format(accuracy * 100))


Decision Tree Accuracy: 84.51%
